In [ ]:
#first lets fetch the data
#we proceed by Downloading examples of spam and ham from Apache SpamAssassin’s public dataset,https://spamassassin.apache.org/old/publiccorpus/ below:
import os
import urllib.request as ul
import tarfile
DOWNLOAD = "https://spamassassin.apache.org/old/publiccorpus/"
SPAM = DOWNLOAD+"20030228_spam.tar.bz2"
HAM = DOWNLOAD+"20030228_easy_ham.tar.bz2"
SPAM_PATH = os.path.join("dataset","spam")
HAM_PATH = os.path.join("dataset","ham")

def retrieve_dataset(spam=SPAM,ham=HAM,spam_path=SPAM_PATH,ham_path=HAM_PATH):
    for path in ((spam_path,ham_path)):
        if not os.path.isfile(path):
            os.makedirs(path)
    for filename,url,extractpath in [("ham.tar.bz2",ham,ham_path),("spam.tar.bz2",spam,spam_path)]:
        archive_path = os.path.join("dataset",filename)
        if not os.path.isfile(archive_path):
            ul.urlretrieve(url,archive_path)
        with tarfile.open(archive_path) as tar_bz2_file:
            tar_bz2_file.extractall(path=extractpath)

In [84]:
retrieve_dataset()

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'dataset\\spam'

In [ ]:
#Next, lets Load filenames from extracted spam and ham directories
spam_dir = os.path.join(SPAM_PATH,"spam")
ham_dir = os.path.join(HAM_PATH,"easy_ham")
spam_filenames = [filename for filename in sorted(os.listdir(spam_dir))]
ham_filenames = [filename for filename in sorted(os.listdir(ham_dir))]

In [64]:
print(len(spam_filenames))
print(len(ham_filenames))

501
2501


In [ ]:
#Now, lets prepare functions to load raw emails into Python objects
import email as e
import email.policy as ep
def load_email(is_spam,filename,spam_path = SPAM_PATH,ham_path = HAM_PATH ):
    if is_spam:
        directory = "spam"
        with open(os.path.join(spam_path,directory,filename),"rb") as f:
            return e.parser.BytesParser(policy = ep.default).parse(f)
    else:
        directory = "easy_ham"
        with open(os.path.join(ham_path,directory,filename),"rb") as f:
            return e.parser.BytesParser(policy = ep.default).parse(f)

In [66]:
ham_emails = [load_email(is_spam=False,filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True,filename=name) for name in spam_filenames]


In [ ]:
# Prepare functions to load raw emails into Python objects
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [ ]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


In [ ]:
#Some emails are actually multipart, with images and attachments (which can have their own attachments). Let's look at the various types of structures we have:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

In [70]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [71]:
structures_counter(ham_emails).most_common()

[('text/plain', 2409),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [ ]:
#It seems that the ham emails are more often plain text, while spam has quite a lot of HTML. Moreover, quite a few ham emails are signed using PGP, while no spam is. In short, it seems that the email structure is useful information to have.
structures_counter(spam_emails).most_common()

[('text/plain', 219),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

In [73]:
for header, value in spam_emails[0].items():
    print(header,":",value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

In [ ]:
#okay now we are ready to split it into a training set and a test set:
import numpy as np
from sklearn.model_selection import train_test_split
X = np.array(ham_emails+spam_emails,dtype = object)
y = np.array([0]*len(ham_emails)+[1]*len(spam_emails))

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#Okay, let's start writing the preprocessing functions. First, we will need a function to convert HTML to plain text. Arguably the best way to do this would be to use the great BeautifulSoup library
# The following function first drops the <head> section, then converts all <a> tags to the word HYPERLINK, then it gets rid of all HTML tags, leaving only the plain text. For readability, it also replaces multiple newlines with single newlines, and finally it unescapes html entities (such as &gt; or &nbsp;)
from bs4 import BeautifulSoup
import re
from html import unescape
def html_to_text(html: str)->str:
    soup = BeautifulSoup(html,"html.parser")
    if soup.head:
        soup.head.decompose()

    for a in soup.find_all("a"):
        a.replace_with(" HYPERLINK")

    text = soup.get_text(separator=" ")
    text = re.sub(r'(\s*\n)+', "\n", text, flags=re.M)


In [ ]:
#Great! Now let's write a function that takes an email as input and returns its content as plain text, whatever its format is:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if ctype not in ("text/plain","text/html"):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_text(html)

In [ ]:
#now we create a custom transformer to convert emails to word count dictionaries

from sklearn.base import BaseEstimator, TransformerMixin
import urlextract
import re
from nltk.stem import PorterStemmer
from collections import Counter
url_extractor = urlextract.URLExtract()
stemmer = PorterStemmer()

class emailToWordCountTransformer(BaseEstimator,TransformerMixin):
    def __init__(self,strip_headers=True,lower_case=True,remove_punctuation=True,replace_url=True,replace_numbers=True,stemming=True):
        self.strip_headers=strip_headers
        self.lower_case=lower_case
        self.remove_punctuation=remove_punctuation
        self.replace_url=replace_url
        self.replace_numbers=replace_numbers
        self.stemming=stemming
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        x_transformed = []
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_url and url_extractor is not None:
                urls = list(url_extractor.find_urls(text))
                for url in urls:
                    text = text.replace(url,"URL")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+',' ',text,flags = re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            x_transformed.append(word_counts)
        return np.array(x_transformed)
            

In [ ]:
#Now we have the word counts, and we need to convert them to vectors. For this, we will build another custom transformer to convert word counts into sparse feature vectors
from scipy.sparse import csr_matrix
from collections import Counter
class wordToVectorTransformer(TransformerMixin,BaseEstimator):
    def __init__(self,vocabulary_size = 10000):
        self.vocabulary_size = vocabulary_size
    def fit(self,X,y=None):
        word_counter = Counter()
        for word_count in X:
            for word,count in word_count.items():
                word_counter[word] += min(count,10)
        most_common = word_counter.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word:index+1 for index,(word,count) in enumerate(most_common)}
        return self
    def transform(self,X,y=None):
        rows = []
        cols = []
        data = []
        for row,word_count in enumerate(X):
            for word,count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word,0))
                data.append(count)
        return csr_matrix((data,(rows,cols)),shape = (len(X), self.vocabulary_size+1))

In [ ]:
#Build a full preprocessing pipeline
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("email_to_wordcount", emailToWordCountTransformer()),
    ("wordcount_to_vector", wordToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(x_train)

In [ ]:
#Select and train a classification model (Logistic Regression)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

[CV] END ................................ score: (test=0.975) total time=   0.1s
[CV] END ................................ score: (test=0.965) total time=   0.2s
[CV] END ................................ score: (test=0.979) total time=   0.2s


np.float64(0.9729270703287557)

In [82]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = preprocess_pipeline.transform(x_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred)))

Precision: 91.82%
Recall: 97.12%
